## Authenticate to Google Cloud

You need to log into Google Cloud with a valid account before running the notebook. Your account has to have permissions to create Spanner databases. 

You also need a project with Spanner enabled.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('You are authenticated!')

## Install the Spanner Python API

You will be prompted to restart the runtime. Do so.

In [ ]:
! pip install --upgrade google-cloud-spanner

## Set the following variables 

You at least need to change the Project ID variable to your project's ID. You can change the other variables if you like. 

In [ ]:
project_id = 'roi-spanner'
instance_id = 'spanner-interleaved-ex'
processing_units = 100
database_id = 'pets-interleaved'

OPERATION_TIMEOUT_SECONDS = 240

print("Done")

## Create a Spanner Instance

In [ ]:
import time
from google.cloud import spanner

def create_instance_with_processing_units(instance_id, processing_units):
    """Creates an instance."""
    spanner_client = spanner.Client(project=project_id)

    config_name = "{}/instanceConfigs/regional-us-central1".format(
        spanner_client.project_name
    )

    instance = spanner_client.instance(
        instance_id,
        configuration_name=config_name,
        display_name="spanner-instance-interleaved",
        processing_units=processing_units,
    )

    # create() returns a long-running operation
    operation = instance.create()

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print(
        "Created instance {} with {} processing units".format(
            instance_id, instance.processing_units
        )
    )

# Call the function
create_instance_with_processing_units(instance_id, processing_units)

## Create the Pets Database

In [ ]:
def create_database(instance_id, database_id):
    """Creates a database and tables for sample data."""
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)

    database = instance.database(
        database_id,
    )

    # create() returns a long-running operation
    operation = database.create()

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print("Created database {} on instance {}".format(database_id, instance_id))

# Call the function
create_database(instance_id, database_id)


## The following functi0n will run DDL statements to build the database schema

In [ ]:
def run_ddl_statement(instance_id, database_id, ddl):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    operation = database.update_ddl(
        [ddl]
    )

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print("Ran statement: {}".format(ddl))


## Create the Owners table

In [ ]:
ddl = """CREATE TABLE Owners (
                  OwnerID STRING(36) NOT NULL,
                  OwnerName STRING(MAX) NOT NULL
               ) PRIMARY KEY (OwnerID)"""



run_ddl_statement(instance_id, database_id, ddl)

## Create the Pets table

Note: the pets table is interleaved with the Owners table. This means each pet is stored with its owner. The primary key of the Pets table uses both OwnerID and PetID. 

In [ ]:
ddl = """CREATE TABLE Pets (
                  OwnerID STRING(36) NOT NULL, 
                  PetID STRING(MAX) NOT NULL,     
                  PetType STRING(MAX) NOT NULL,
                  PetName STRING(MAX) NOT NULL,
                  Breed STRING(MAX) NOT NULL,
              ) PRIMARY KEY (OwnerID,PetID),
                INTERLEAVE IN PARENT Owners ON DELETE CASCADE"""


run_ddl_statement(instance_id, database_id, ddl)

## Display the Database Schema

In [ ]:
def get_database_ddl(instance_id, database_id):
    """Gets the database DDL statements."""
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)
    ddl = spanner_client.database_admin_api.get_database_ddl(database=database.name)

    print("Retrieved database DDL for {}".format(database_id))

    for statement in ddl.statements:
        print(statement)


get_database_ddl(instance_id, database_id)

## Add Some Test Records

In [ ]:
import uuid

def insert_data(instance_id, database_id):
    doug_id = str(uuid.uuid4())
    john_id = str(uuid.uuid4())
    sue_id = str(uuid.uuid4())

    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    with database.batch() as batch:
        batch.insert(
            table="Owners",
            columns=("OwnerID", "OwnerName"),
            values=[
                (doug_id, u"Doug"),
                (john_id, u"John"),
                (sue_id, u"Sue"),
            ],
        )

        batch.insert(
            table="Pets",
            columns=("PetID", "OwnerID", "PetType", "PetName", "Breed"),
            values=[
                (str(uuid.uuid4()), doug_id, u"Dog", u"Noir", u"Schnoodle"),
                (str(uuid.uuid4()), doug_id, u"Dog", u"Bree", u"Mutt"),
                (str(uuid.uuid4()), doug_id, u"Cat", u"Tom", u"Alley"),
                (str(uuid.uuid4()), john_id, u"Dog", u"Duke", u"GoldenDoodle"),\
                (str(uuid.uuid4()), john_id, u"Dog", u"Sparky", u"Poodle"),
                (str(uuid.uuid4()), john_id, u"Turtle", u"Cuff", u"Box"),
                (str(uuid.uuid4()), john_id, u"Turtle", u"Link", u"Box"),
                (str(uuid.uuid4()), sue_id, u"Cat", u"Cleo", u"Domestic"),
            ],
        )

    print("Inserted data.")


insert_data(instance_id, database_id)

## Runs the Query Passed as an Argument

In [ ]:
def run_query(sql):
  # Instantiate a client.
  spanner_client = spanner.Client(project=project_id)

  # Get a Cloud Spanner instance by ID.
  instance = spanner_client.instance(instance_id)

  # Get a Cloud Spanner database by ID.
  database = instance.database(database_id)

  # Execute a simple SQL statement.
  with database.snapshot() as snapshot:
      results = snapshot.execute_sql(sql)
      for row in results:
        print(row)


## Query with a Join

In [ ]:
sql = """SELECT Owners.OwnerID, OwnerName, PetName, PetType, Breed 
         FROM Owners 
         JOIN Pets ON Owners.OwnerID = Pets.OwnerID;"""

run_query(sql)

Query with Nested Array

In [ ]:
sql = """SELECT OwnerName,
  ARRAY(SELECT AS STRUCT PetName, PetType, Breed FROM Pets WHERE OwnerID = '54b4a7cd-e8cd-4ce6-9775-ac149460fc96') as Pets,
FROM Owners WHERE OwnerID = '54b4a7cd-e8cd-4ce6-9775-ac149460fc96';
"""

run_query(sql)

## Query with Aggregated Array

In [ ]:
sql = """SELECT OwnerName, 
         Array_Agg(STRUCT(PetName, PetType, Breed)) as Pets
         FROM Owners 
         JOIN Pets ON Owners.OwnerID = Pets.OwnerID
         GROUP BY OwnerName;"""

run_query(sql)

## Delete All the Data

In [ ]:
def delete_data_with_dml(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    def delete_owners(transaction):
        
        row_ct = transaction.execute_update(
            "DELETE FROM Owners WHERE true = true"
        )

        print("{} record(s) deleted.".format(row_ct))

    database.run_in_transaction(delete_owners)

# Call the function
delete_data_with_dml(instance_id, database_id)


## Delete the Database

In [ ]:
def delete_database(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)
    
    database.drop()

    print("{} database dropped".format(database_id))

    

# Call the function
delete_database(instance_id, database_id)

## Delete the Instance

In [ ]:
def delete_spanner_instance(instance_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    instance.delete()

    print("{} instance deleted".format(instance_id))

# Call the function
delete_spanner_instance(instance_id)